In [167]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [168]:
##Load the dataset
data=pd.read_csv("Churn_Modelling.csv"
                 )
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [169]:
## Preprocess the data
# drop the irrelevant data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [170]:
data.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [171]:
##Encode categorical variable
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head(6)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1


In [172]:
##Use onehotencoder for geographical column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [173]:
print(geo_encoder)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>
  Coords	Values
  (0, 0)	1.0
  (1, 2)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 2)	1.0
  (5, 2)	1.0
  (6, 0)	1.0
  (7, 1)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 2)	1.0
  (12, 0)	1.0
  (13, 0)	1.0
  (14, 2)	1.0
  (15, 1)	1.0
  (16, 1)	1.0
  (17, 2)	1.0
  (18, 2)	1.0
  (19, 0)	1.0
  (20, 0)	1.0
  (21, 2)	1.0
  (22, 2)	1.0
  (23, 0)	1.0
  (24, 0)	1.0
  :	:
  (9975, 1)	1.0
  (9976, 0)	1.0
  (9977, 0)	1.0
  (9978, 0)	1.0
  (9979, 0)	1.0
  (9980, 2)	1.0
  (9981, 1)	1.0
  (9982, 1)	1.0
  (9983, 0)	1.0
  (9984, 1)	1.0
  (9985, 0)	1.0
  (9986, 1)	1.0
  (9987, 2)	1.0
  (9988, 0)	1.0
  (9989, 2)	1.0
  (9990, 1)	1.0
  (9991, 0)	1.0
  (9992, 2)	1.0
  (9993, 0)	1.0
  (9994, 0)	1.0
  (9995, 0)	1.0
  (9996, 0)	1.0
  (9997, 0)	1.0
  (9998, 1)	1.0
  (9999, 0)	1.0


geo_encoder

In [174]:
onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_encoder=geo_encoder.toarray()

In [175]:
geo_encoded_df = pd.DataFrame(geo_encoder,columns=['Geography_France', 'Geography_Germany', 'Geography_Spain'])


In [176]:
geo_encoder.shape

(10000, 3)

In [177]:
#COmbine onehotencoded columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [178]:
data.head(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [179]:
## Save the encoders and scaler to nuse in future for deployment
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)


In [180]:
##Dividing the dataset into dependent and independent features
X=data.drop('Exited',axis=1)
y=data['Exited']
#Split the data into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
##Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [181]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [182]:
##ANN Implementation

##ANN Implementation

In [183]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime



In [184]:
X_train.shape[1]

12

In [185]:
##Build our ANN model
model=Sequential([Dense(64,activation='relu',input_shape=(X_train.shape[1],)),##HL1 connected with imput layer
                  Dense(32,activation='relu'),##HL2 ##No need to give input from here
                  Dense(1,activation='sigmoid')  ##OUtput ##BInary output
                  ]

)

In [186]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 64)                832       
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [187]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
tensorflow.keras.losses.BinaryCrossentropy()

In [188]:
##COmpile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [189]:
##Setup the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit_latest"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [190]:
##setup Early stopping 
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [191]:
###Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.3935 - accuracy: 0.8386 - val_loss: 0.3549 - val_accuracy: 0.8560
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3516 - accuracy: 0.8556 - val_loss: 0.3508 - val_accuracy: 0.8560
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3459 - accuracy: 0.8606 - val_loss: 0.3395 - val_accuracy: 0.8595
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3434 - accuracy: 0.8619 - val_loss: 0.3447 - val_accuracy: 0.8580
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3407 - accuracy: 0.8626 - val_loss: 0.3610 - val_accuracy: 0.8500
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3381 - accuracy: 0.8622 - val_loss: 0.3371 - val_accuracy: 0.8595
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3378 - accuracy: 0.8616 - val_loss: 0.3392 - val_accuracy: 0.8545

In [192]:
##SAve model as h5(for keras)
model.save('model.h5')

c:\Users\19408\anaconda3\envs\mine\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [193]:
##Load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [194]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 15168), started 0:08:16 ago. (Use '!kill 15168' to kill it.)